IMU data sampling

In [4]:
import csv
import random
import numpy as np
import os
import shutil
import cv2

In [266]:
240729

113848
165940
174101

240730

160424
161728
163508

240807

131207

240809

172448

174101

In [418]:
day = '240809'
can_Time = '20240809_172448'

print(day, can_Time)

240809 20240809_172448


In [419]:
cansat_data = []
#directory = f"D:\cansat_log\cantest_{day}"
directory = f"D:/cansat_log/cantest_{day}/cansat_test_{can_Time}"
with open(f"{directory}/cansat_log_{can_Time}.csv", 'r') as f:
    rdr = csv.reader(f)
    for line in rdr:
        cansat_data.append(line)
print(len(cansat_data))

89131


In [270]:
"""
cansat_data = []
directory = f"D:\cansat_log\cantest_{day}"
#directory = f"D:\cansat_log\cantest_{day}\cansat_test_{can_Time}"
with open("D:\cansat_log\cantest_240410\labels.txt", 'r') as f:
    rdr = f.readlines()
    
    for line in rdr:
        cansat_data.append(line[:-2].split())

print(cansat_data)
"""

'\ncansat_data = []\ndirectory = f"D:\\cansat_log\\cantest_{day}"\n#directory = f"D:\\cansat_log\\cantest_{day}\\cansat_test_{can_Time}"\nwith open("D:\\cansat_log\\cantest_240410\\labels.txt", \'r\') as f:\n    rdr = f.readlines()\n    \n    for line in rdr:\n        cansat_data.append(line[:-2].split())\n\nprint(cansat_data)\n'

In [271]:
"""
#print(cansat_data)
imu_data = []
for line in cansat_data:

    imu_data.append([float(line[0]), float(line[1]), float(line[2])])

print(len(imu_data))
print(imu_data)
"""

'\n#print(cansat_data)\nimu_data = []\nfor line in cansat_data:\n\n    imu_data.append([float(line[0]), float(line[1]), float(line[2])])\n\nprint(len(imu_data))\nprint(imu_data)\n'

In [420]:
imu_data = []
camera_time = []
cnt0=0
for line in cansat_data:
    if line[0] == 'Picture' and line[1][49]=='0':
        camera_time.append(line[1][59:-4])
        if line[1][59:-4] == '20240809_172615167199':
            start = len(imu_data)
            print("start")
        if line[1][59:-4] == '20240809_173815406997':
            end = len(imu_data)
            print("end")
    if line[0] == 'IMU_DATA':
        r = float(line[1][1:])
        p = float(line[2])
        y = float(line[3])
        
        imu_data.append([r,p,y])
    

print(len(imu_data))
print(len(camera_time))
print(camera_time[0])
print(start, end)

start
end
22054
22055
20240809_172457331384
1761 19420


In [421]:
sample_len = (end-start)//3100
print(sample_len)

5


In [422]:
samples = []
#s=random.randint(500, 600)
s=start + random.randint(50, 100)
e=end
for _ in range(sample_len):
    length = random.randint(2700, 3300)
    if s+length > e:
        length = e - s
    samples.append((s, length))
    gap = random.randint(100,300)
    s += length + gap

print(samples)

[(1835, 2746), (4876, 3076), (8154, 2911), (11342, 2707), (14331, 3091)]


In [423]:
def sampling_0410(sample,i):

    directory_sample = f"D:/cansat_log_sampleing/cantest_{day}/cansat_sample_{can_Time}_{i+1}"
    os.makedirs(f"{directory_sample}/Camera_0", exist_ok=True)
    os.makedirs(f"{directory_sample}/Camera_1", exist_ok=True)

    start,data_len = sample
    sample_data = []

    R = imu_data[start][0]
    P = imu_data[start][1]
    Y = imu_data[start][2]

    for i in range(start,start+data_len):
        line = imu_data[i]
        r = line[0] - R
        if r<-180:
            r+=360
        elif r>180:
            r-=360

        p = line[1] - P
        if p<-90:
            p+=180
        elif p>90:
            p-=180

        y = line[2] - Y
        if y<-180:
            y+=360
        elif y>180:
            y-=360
            
        sample_data.append([r, p, y])

        source = f"{directory}\Camera_Image\Camera_0\Camera0_{camera_time[i]}.jpg"
        directory_img = f"{directory_sample}/Camera_0/{i+1-start}.jpg"
        shutil.copy(source, directory_img)

        source = f"{directory}\Camera_Image\Camera_1\Camera1_{camera_time[i]}.jpg"
        directory_img = f"{directory_sample}/Camera_1/{i+1-start}.jpg"
        shutil.copy(source, directory_img)
    
    return sample_data



In [424]:
for i in range(sample_len):
    sample = samples[i]
    sample_data = sampling_0410(sample,i)
    directory_sample = f"D:/cansat_log_sampleing/cantest_{day}/cansat_sample_{can_Time}_{i+1}"
    os.makedirs(directory_sample, exist_ok=True)
    filename = f'{directory_sample}/{can_Time}_imu_data_sample{i+1}.txt'
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter=' ')
        writer.writerows(sample_data)


Make MP4

In [190]:
def img_copy():

    for i in range(len(camera_time)):
        
        source = f"{directory}\Camera_Image\Camera_1\Camera1_{camera_time[i]}.jpg"
        directory_img = f"C:/Users/user/Desktop/cansat/cansat_goheung/camera 1/{i}.jpg"
        shutil.copy(source, directory_img)
    
    return sample_data



In [ ]:
img_copy()

In [12]:
def create_mp4(directory_mp4, cam_num, fps):
    image_folder = f"{directory_mp4}/camera{cam_num}"
    video_filename = f"{directory_mp4[-17:]}_cam{cam_num}.mp4"
    
    start_index = 0
    end_index = 4540


    image_files = [f"{image_folder}/{str(i).zfill(6)}.jpg" for i in range(start_index, end_index + 1)]

    frame = cv2.imread(image_files[0])
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video = cv2.VideoWriter(video_filename, fourcc, fps, (width, height)) 

    for image_file in image_files:
        img = cv2.imread(image_file)
        video.write(img)

    video.release()
    cv2.destroyAllWindows()

    print(f"Video created successfully: {video_filename}")


In [13]:
directory_mp4 = "D:\KITTY"
directory_mp4 = directory_mp4.replace('\\', '/')
fps = 30

print(directory_mp4)

create_mp4(directory_mp4, 0, fps)
create_mp4(directory_mp4, 1, fps)

D:/KITTY
!
!
!
D:/KITTY/camera0/000000.png
!
Video created successfully: D:/KITTY_cam0.mp4
!
!
!
D:/KITTY/camera1/000000.png
!
Video created successfully: D:/KITTY_cam1.mp4
